In [1]:
!pip install accelerate
!pip install modelscope

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive')
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NLP_proj'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['train.csv', 'predict.csv', '.ipynb_checkpoints']


In [2]:
DATA_PATH = GOOGLE_DRIVE_PATH
print(os.listdir(DATA_PATH))
file_name = 'train.csv'
file_path = os.path.join(DATA_PATH, file_name)
print(file_path)

['train.csv', 'predict.csv', '.ipynb_checkpoints']
drive/My Drive/NLP_proj/train.csv


In [3]:
import json
import torch
import os
import re
import pandas as pd
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
import accelerate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#cache_dir = GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', "Mistral-7B-Instruct-v0.2")
#model_dir = snapshot_download('AI-ModelScope/Mistral-7B-Instruct-v0.1', revision='v1.0.0',cache_dir=cache_dir)

2024-06-29 12:49:56,223 - modelscope - INFO - PyTorch version 2.3.0+cu121 Found.
2024-06-29 12:49:56,228 - modelscope - INFO - TensorFlow version 2.15.0 Found.
2024-06-29 12:49:56,230 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-06-29 12:49:56,274 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 4e0ef0a3f41415ecc99228a124fc2939 and a total number of 980 components indexed


In [4]:
model_dir = GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', "Mistral-7B-Instruct-v0.1")
print(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
pad_token_id = tokenizer.eos_token_id

drive/My Drive/Mistral-7B-Instruct-v0.1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

hyperparameters = {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 0.7,
    "top_k": 80,
    "top_p": 0.9,
    "num_return_sequences": 1,
    "pad_token_id": pad_token_id
}

system_prompt = "You are an expert in Algebra. Lets think through these questions step by step to ensure we get the " \
                "correct answer. There are only 5 options to choose from A,B,C,D or E. Ensure that your answer " \
                "follows this format: Answer: "

In [6]:
def extract_answer_final(generated_text, question_range):
    # Construct the regex pattern with the variable using an f-string
    pattern = rf"Answer:\s*([{question_range}])"
    match = re.search(pattern, generated_text)
    if match:
        # If a match is found, return the letter and True for success
        return match.group(1), True
    else:
        # If no match is found, return None and False to indicate failure
        return None, False

In [7]:
# Conservative Sampling
hyperparameters_1 = {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 1,
    "pad_token_id": pad_token_id
}

# Middleground Sampling
hyperparameters_2 = {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 0.7,
    "top_k": 80,
    "top_p": 0.9,
    "num_return_sequences": 1,
    "pad_token_id": pad_token_id
}

# Creative Sampling
hyperparameters_3 = {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 1.0,
    "top_k": 100,
    "top_p": 0.85,
    "num_return_sequences": 1,
    "pad_token_id": pad_token_id
}
hyperparameters_list = [hyperparameters_1, hyperparameters_2, hyperparameters_3]

In [8]:
df = pd.read_csv(file_path)[: 50]

In [11]:
for row_index, row in df.iterrows():
    question = row['question'] + row['options']
    combined_question = f"{system_prompt}. {question}"
    max_attempts = 6  # Define a maximum number of attempts to avoid infinite loops
    attempts = 0
    answers = []
    success = False

    while attempts < max_attempts:                                  # try max_attempts times and use majority vote
        hyperparameters = hyperparameters_list[attempts % 3]        # Select hyperparameters based on attempt
        output = pipe(combined_question, **hyperparameters)
        generated_text = output[0]['generated_text']
        print(f"\nText: {generated_text}\n Hyperparameters: {hyperparameters}")
        correct_option, success = extract_answer_final(generated_text, "A-E")

        if success:                                                 # if success, append answer to list of answers
            print(f"\nQuestion ID {row_index}, Attempt {attempts}: Correct Option - {correct_option}")
            answers.append(correct_option)
        else:
            print(f"Attempt {attempts} failed, retrying...")

        attempts += 1

    if answers:
        most_common_answer, count = Counter(answers).most_common(1)[0]
        if count > 1:                                               # if there is a majority, add to csv
            print(f"\nQuestion ID {row_index}: Majority Option - {most_common_answer}")
            df.at[row_index, 'y'] = most_common_answer
            continue

    # if failed to get majority, make a final attempt with hyperparameters_2
    print(f"No clear majority for question ID {row_index}, making a final attempt with hyperparameters_2.")
    final_output = pipe(combined_question, **hyperparameters_2)
    final_generated_text = final_output[0]['generated_text']
    final_correct_option, final_success = extract_answer_final(final_generated_text, "A-E")

    if final_success:
        print(f"Final Attempt Success: Correct Option - {final_correct_option}")
        df.at[row_index, 'y'] = final_correct_option
    else:
        print(f"Final attempt failed to parse a valid answer for question ID {row_index}.")
        df.at[row_index, 'y'] = "Error"

# Save the DataFrame with the updated 'y' column to a new CSV file
output_csv_path = predict_path = os.path.join(DATA_PATH, 'predict.csv')
df.to_csv(output_csv_path, index=False)
print(f"Updated CSV saved to {output_csv_path}")


Text: You are an expert in Algebra. Lets think through these questions step by step to ensure we get the correct answer. There are only 5 options to choose from A,B,C,D or E. Ensure that your answer follows this format: Answer: . Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?['A)21', 'B)21.5', 'C)22', 'D)22.5', 'E)23']
Answer: C)22
 Hyperparameters: {'do_sample': True, 'max_new_tokens': 100, 'temperature': 0.5, 'top_k': 50, 'top_p': 0.95, 'num_return_sequences': 1, 'pad_token_id': 2}

Question ID 0, Attempt 0: Correct Option - C

Text: You are an expert in Algebra. Lets think through these questions step by step to ensure we get the correct answer. There are only 5 options to choose from A,B,C,D or E. Ensure that your answer follows this format: Answer: . Two friends plan to walk along a 43-km trail, starting

In [12]:
df = pd.read_csv(output_csv_path, encoding='utf-8')
equals = df['correct'].eq(df['y'])
equal_count = equals.sum()
correct_rate=equal_count/len(df)
print(correct_rate)

0.26
